In [1]:
import tkinter as tk
import matplotlib
%matplotlib qt
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import numpy as np
from tkinter.filedialog import asksaveasfilename
import csv

In [80]:
class SiteLocator(tk.Tk):
    """
    Provide attributes and methods for creating a GUI-based site location app.
    Parameters
    ----------
    None    
    """
    def __init__(self) -> None:
        super().__init__()  # attributes of tk.Tk
        # Window properties
        self.title("Industrial Site Locator")
        self.geometry("1000x800")
        # Raster list
        self.rasters = [np.genfromtxt("geology.csv", dtype=int, delimiter=","),\
            np.genfromtxt("population.csv", dtype=int, delimiter=","), np.genfromtxt("transport.csv", dtype=int, delimiter=",")]
        # Map variables
        self.geoweight = tk.IntVar(self, value=100)
        self.popweight = tk.IntVar(self, value=100)
        self.transweight = tk.IntVar(self, value=100)
        # Top 10% blue colormap
        self.blue_cmap = self.cmap_creator()
        # Blue checkbox variables
        self.blue = tk.IntVar(self, value=0)
        self.plotcolour = "gray"
        # Currently displayed raster variable
        self.current_raster = []  # initialised as empty list
        # Figure variable
        self.fig = matplotlib.figure.Figure(figsize=(5, 8))  # do not use plt figure, that pops up second window
        self.ax = self.fig.add_subplot(111, visible=False)  # do not show until first map shows up
        self.is_ax_mapped = False
        # Frames
        self.infoframe = tk.Frame(self, bg="white")
        self.infoframe.pack(side="left", expand=1, fill="both")
        self.mapframe = tk.Frame(self, bg="white")
        self.mapframe.pack(side="right", expand=1, fill="both")
        # Text labels
        # Title of the application
        # Bold adapted from https://stackoverflow.com/a/46495200/18668457
        self.maintext = tk.Label(self.infoframe, text="Industrial Site\nLocator", bg="white", borderwidth=0, highlightthickness=0, font=("Arial", 30, "bold"), fg="#31a354")
        self.maintext.place(relx=0.1, rely=0.1, relheight=0.15, relwidth=0.8, anchor="nw")
        # Explanatory text
        self.explanation = tk.Label(self.infoframe, text="Use the scrollbars to adjust the importance of factors.\nYou can display and save the maps to .csv files.", bg="white", borderwidth=0, highlightthickness=0, font=("Arial", 10))
        self.explanation.place(relx=0.1, rely=0.25, relheight=0.1, relwidth=0.8, anchor="nw")
        # Scales
        # Highlightthickness adapted from https://stackoverflow.com/a/4311134/18668457
        # Geology
        self.geoscale = tk.Scale(self.infoframe, from_=0, to=100, orient="horizontal", variable=self.geoweight, label="Geology", bg="white", troughcolor="#a1d99b", bd=0, highlightthickness=0, activebackground="white", font=("Arial", 10, 'bold'))
        self.geoscale.place(relx=0.1, rely=0.4, relheight=0.1, relwidth=0.8, anchor="nw")
        # Population
        self.popscale = tk.Scale(self.infoframe, from_=0, to=100, orient="horizontal", variable=self.popweight, label="Population", bg="white", troughcolor="#a1d99b", bd=0, highlightthickness=0, activebackground="white", font=("Arial", 10, 'bold'))
        self.popscale.place(relx=0.1, rely=0.5, relheight=0.1, relwidth=0.8, anchor="nw")
        # Transport
        self.transscale = tk.Scale(self.infoframe, from_=0, to=100, orient="horizontal", variable=self.transweight, label="Transport", bg="white", troughcolor="#a1d99b", bd=0, highlightthickness=0, activebackground="white", font=("Arial", 10, 'bold'))
        self.transscale.place(relx=0.1, rely=0.6, relheight=0.1, relwidth=0.8, anchor="nw")
        # Display button
        self.displaybutton = tk.Button(self.infoframe, text="Display map", command=self.display, bg="#31a354", bd=1, fg="white", font=("Arial", 10, "bold"))
        self.displaybutton.place(relx=0.1, rely=0.8, relheight=0.1, relwidth=0.35, anchor="nw")
        # Saving functionality
        self.savebutton = tk.Button(self.infoframe, text="Save map", command=self.save, bg="red", bd=1, fg="white", font=("Arial", 10, "bold"))
        self.savebutton.place(relx=0.55, rely=0.8, relheight=0.1, relwidth=0.35, anchor="nw")
        # Blue checkbox
        self.bluecheckbox = tk.Checkbutton(self.infoframe, text="Display top 10% in blue", var=self.blue, onvalue=1, offvalue=0, command=self.turn_blue, bg="white", bd=0, highlightthickness=0, font=("Arial", 10), activebackground="white")
        self.bluecheckbox.place(relx=0.2, rely=0.7, relheight=0.05, relwidth=0.6, anchor="nw")
        # Canvas
        self.canvas = matplotlib.backends.backend_tkagg.FigureCanvasTkAgg(self.fig, master=self.mapframe)
        self.canvas._tkcanvas.place(relx=0, rely=0, relheight=1, relwidth=1, anchor="nw")

    # Triggers display on button click
    def display(self) -> None:
        """Triggers the functions needed for creating, weighting, and showing the map."""
        weights = self.weight_reader()  # get weights
        avg_raster = self.raster_overlayer(self.rasters, weights)  # get averaged raster
        self.current_raster = avg_raster  # set current raster
        self.shower(avg_raster)  # display current raster

    # Draws map on the canvas
    def shower(self, raster: list) -> None:
        """
        Shows the map image on an axes subplot with the appropriate colour scheme.
        Parameters
        ----------
        raster : list
            The final, weighted raster.
        """
        self.ax.clear()
        self.ax.set_visible(True)  # set subplot visible
        self.is_ax_mapped = True
        img = self.ax.imshow(raster, cmap=self.plotcolour)  # creates image to plot
        self.canvas.draw()  # draws image on canvas

    # Read rasters at the beginning of class and return them in a list
    def raster_reader(self) -> list:
        """
        Reads the three input rasters (geology, population, transport) into a list of numpy arrays.
        Parameters
        ----------
        None

        Returns
        -------
        list
            List of rasters (numpy ndarrays)
        """
        ras_list = [np.genfromtxt("geology.csv", dtype=int, delimiter=","),\
            np.genfromtxt("population.csv", dtype=int, delimiter=","), np.genfromtxt("transport.csv", dtype=int, delimiter=",")]
        return ras_list

    # Reads values of the horizontal scalebars to a list
    def weight_reader(self) -> list:
        """
        Reads the weight variables to each map and returns them in a list.
        Parameters
        ----------
        None

        Returns
        -------
        list
            List of weights corresponding to rasters
        """
        weights = [self.geoweight.get(), self.popweight.get(), self.transweight.get()]
        return weights

    # Calculates the average of the three rasters, returns it ranging to 255 as integers
    def raster_overlayer(self, rasters: list, weights: list) -> list:
        """
        Calculates the weighted average of the three rasters.
        Parameters
        ----------
        rasters : list
            List of the three input rasters
        weights : list
            List of the three weights corresponding to rasters
        
        Returns
        -------
        np.ndarray
            Final, averaged, rounded raster (numpy ndarray)
        """
        # Zip function from https://stackoverflow.com/a/1663826/18668457
        # np.multiply from https://numpy.org/doc/stable/reference/generated/numpy.multiply.html
        weighted_rasters = [np.multiply(raster, weight) for raster, weight in zip(rasters, weights)]
        # Take average of the weighted rasters
        avg_ras = np.sum(weighted_rasters, axis=0) / len(weighted_rasters)
        # Amax adapted from https://numpy.org/doc/stable/reference/generated/numpy.amax.html
        # Astype adapted from https://appdividend.com/2020/05/06/how-to-convert-numpy-float-to-int-array-in-python/#:~:text=To%20convert%20numpy%20float%20to%20int%20array%20in%20Python%2C%20use,it%20into%20an%20integer%20array.
        rounded_ras = np.round(np.multiply(avg_ras, 255/np.amax(avg_ras))).astype(int)
        return rounded_ras

    # Save displayed map on button click if it has already been displayed
    def save(self) -> None:
        """Saves the currently displayed raster file to a .csv file to the location specified by the user."""
        if len(self.current_raster) == 0:
            tk.messagebox.showwarning("Industrial Site Locator - warning", "You have to first display the map you want to save!")
        else:
            # Ask user for filename
            filename = asksaveasfilename(defaultextension=".csv", filetypes=[("Comma Separated Values", "*.csv")])
            # Try to save the file
            try:
                with open(filename, "w") as f:
                    writer = csv.writer(f)
                    for line in self.current_raster:
                        writer.writerow(line)
                tk.messagebox.showinfo("Industrial Site Locator - information", "Your map has been saved as {}".format(filename))
            # If the name has not been specified, saving has been cancelled
            except FileNotFoundError:
                pass  # this allows the user to decide on not saving the file without errors returned

    # Create colormap at the beginning of class and return it
    def cmap_creator(self) -> matplotlib.colors.ListedColormap:
        """
        Creates a custom colormap from gray and Blues_r with the top 10% being blue.
        Parameters
        ----------
        None

        Returns
        -------
        matplotlib.colors.ListedColormap
            A colormap object with 256 values, the bottom 90% gray, the top 10% blue gradient colors
        """
        bottom = matplotlib.cm.get_cmap("gray", 256)
        top = matplotlib.cm.get_cmap("Blues_r", 256)
        bluetop = np.vstack((bottom(np.linspace(0, 230/256, 230)), top(np.linspace(0, 0.9, 26))))
        bluetopcm = matplotlib.colors.ListedColormap(bluetop, name="Gray90Blue10")
        return bluetopcm

    # Control the plotting colour of the map on checkbox action
    def turn_blue(self):
        """Sets the plotting colour of the map reacting to a checkbox. If the map is already displayed, automatically changes the color."""
        if self.blue.get() == 1:
            self.plotcolour = self.blue_cmap
        elif self.blue.get() == 0:
            self.plotcolour = "gray"
        if self.is_ax_mapped == True:
            self.display()


In [81]:
# Main function to initialise object and start execution
def main() -> None:
    """Main function, initialises a new SiteLocator object, starts mainloop."""
    root = SiteLocator()  # new SiteLocator object
    root.mainloop()

# When the notebook is run, execute the main function
if __name__ == "__main__":
    main()
